# IBM Quantum Experience Lab

Dan Browne (first written November 2018 - updated April 2021)

For this lab exercise you will learn how to use IBM's QISKIT Python environment to perform quantum computations on IBM's prototype chips, via the IBM Quantum Experience cloud interface.

These notes provide a self-contained introduction to QISKIT, but you can also find the official QISKIT tutorial documentaion at the following links:

https://qiskit.org/learn/

https://qiskit.org/documentation/index.html

The introductory part of these notes were inspired by some of IBM's tutorials.

There is an important point we need to make. QISKIT is **beta** software, under **rapid development**. These notes were written for **QISKIT version 0.19.1** and has been updated for version **0.25.3**.

In particular, many of the QISKIT tutorials you will find online are *incompatible* with the current versions, as the commands and API have evolved quite quickly.

## IBM Graphical Interface - "Quantum Composer"

Before learning QISKIT, I recommend you explore the graphical interface that IBM have created. This is primarily for educational purposes, but has much of the same functionality (in drag and drop web-app form) as the QISKIT python package.

In particular, you need to set up an account on the graphical interface before you will be granted an API key to access the chips from QISKIT.

I recommend that you work through the tutorial notes on the Quantum Composer available on this course Moodle page. The notes were written at a level accessible to undergraduate students, and also introduce some of the basic concepts of quantum circuits.

## Getting started with QISKIT Terra

QISKIT is the name for IBM's Python environment for quantum computing. It can be used to create quantum circuits, and run them on simulators or even on IBM's chips themselves.

Recently, IBM have subdivided QISKIT into different products, each with a Latin element as a name. The basic Python package that we will use in this lab, is now called **QISKIT Terra**.


To get started, let's importing some packages from QISKIT that we wiill use below. This is a good way to check if QISKIT is intalled on your machine:

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, Aer

If you get error messages, it is likely that the correct version of QISKIT is not installed on your machine.

In [ ]:
#This code outputs the version numbers for all  the installed sub-packages of QISKIT. 
# The overall QISKIT version number is displayed about halfway through the list. 

from qiskit import __qiskit_version__
print(__qiskit_version__)


If you need to install QISKIT, you can do so easily via ``pip`` using:

```pip install qiskit```

# Registers and Circuits

The main purpose of QISKIT is to create descriptions of quantum circuits, run the circuit on a prototype quantum computers, (or a classical simulators of a prototype quantum computer), and then receive and process the outcomes of the computation.

QISKIT uses Python objects to represent the key components of a quantum computation:
* A "register" of an integer number of **qubits**, representing the prototype quantum computer.
* A "register" of an integer number of **classical bits**, which will store the outcomes of measurements of those qubits
* the **quantum circuit** that you wish to run on those qubits, and which will return the classical data in the classical register.

We create those objects as follows:

In [ ]:
# Create a Quantum Register with 5 qubits
qr = QuantumRegister(3)

# Create a Classical Register with 5 bits
cr = ClassicalRegister(3)

# Create a Quantum Circuit acting on the qr and cr register
circuit = QuantumCircuit(qr, cr)

Initially our circuit has no quantum gates or measurements in it. 

**Important**: By convention, every qubit in a computation is always initialised in the $|0\rangle$ state.

**Important**: The qubits and classical bits in the circuit are labelled with integers starting from **zero** (following the convention in Python of counting from zero).

We can create a circuit using Python commands which add logic gates and measurements to the circuit. Here are some examples from IBM's introductory tutorial:

In [ ]:
# Pauli X gate (NOT gate) on qubit 0
circuit.x(qr[0])

# Pauli X gate (NOT gate) qubit 1
circuit.x(qr[1])

# Toffoli gate from qubit 0,1 to qubit 2
circuit.ccx(qr[0], qr[1], qr[2])

# CNOT (Controlled-NOT) gate from qubit 0 to qubit 1
circuit.cx(qr[0], qr[1])

# measure gate from qr to cr
circuit.measure(qr, cr)

Each of the above commands has added an instruction (either a gate or a measurement) to the circuit object. 

A nice way to see the circuit that has been built is to use the 'circuit_drawer' visualisation. 

In [ ]:
from qiskit.tools.visualization import circuit_drawer

circuit_drawer(circuit)

This visualisation has 6 lines in it, the 3 top 3 lines represent qubits, while the bottom three double lines represent classical bits. This visualisation will help us to unpack the commands we used to make the circuit.

*Optional* - you can enable fancier graphics for the circuit, using the option `output="mpl"`. This uses `matplotlib` to render a colourful version of the circuit.

A further possibility is to use  the option `output="latex"`.
This will only work if `pylatexenc` is installed on your machine. Install it with the command `pip install pylatexenc`.

In [ ]:
circuit_drawer(circuit,output="mpl")

In [ ]:
circuit_drawer(circuit,output="latex")

### Quantum and Classical Registers

First let's look the **quantum register object** and how the qubits within it are labelled.

In [ ]:
print(qr)

This output tells us that the `qr` object is a register of 3 qubits. The second argument is the name that QISKIT has given the register. By default this is `q0`.

The qubits are labelled like elements of a Python list, E.g. `qr[0]`, `qr[1]` etc.

There is a similar syntax for the classical register object `cr`. Here, the default name is `c0`. The bits are labelled `cr[0]`, `cr[1]` etc.

In [ ]:
print(cr)

### Adding logic gates to the circuit.

We can now unpack the command `circuit.x(qr[0])`. 

This has the syntax of a Python method - the function `x` belongs to the `circuit` object, so we access it by writing `circuit.x`. This function adds a Pauli X gate to our circuit, and takes as its argument the label of the qubit (in this case `qr[0]`) on which the gate acts.

Other quantum gates can be added to circuits in QISKIT using similar syntax. 

**Exercise**: In the above examples, identify the command that adds the CCX or Toffolli gate (https://en.wikipedia.org/wiki/Toffoli_gate) and the CNOT gate (https://en.wikipedia.org/wiki/Controlled_NOT_gate).

When we apply these gate we need to specify the control qubit(s) and target qubit. Which argument refers to the control qubit(s) and which to the target qubit in the QISKIT syntax?

### Measuring qubits

Have another look at the visualisation of the circuit. The "boxes with dials" represent quantum measurements. In particular, each box represents a computational basis measurement, projecting onto $|0\rangle$ with outcome "0" or $|1\rangle$ with outcome '1'.

We note that all 3 qubits are measured, and vertical lines linking the qubits to a classical bit that will store the outcome of the measurement.

This was acheived with a single command:

`circuit.measure(qr, cr)`

This command measures all qubits in register `qr` and stores the outcomes in classical register `cr`.

You can use the same command to measure individual qubits (or any subset of the register specified as a Python tuple), e.g.:

`circuit.measure((qr[0],qr[1]),(cr[0],cr[1]))`



## Hardware primitive gates

IBM's chip implements logic gates via microwave driving of the structures in the chip. This means that only a limited set of gates can be implemented natively. These include CNOT gates (between certain pairs of qubits with a fixed control and target) and single qubit native gates.

IBM calls the native single qubit gates $u_1$, $u_2$, and $u_3$:

https://qiskit.org/textbook/ch-states/single-qubit-gates.html#generalU3

We will only consider $u_1$ here. This gate is a simple phase gate, and takes one parameter:

$$ u_1(\lambda)=\begin{bmatrix}1 & 0\\0 & \exp(i \lambda)\end{bmatrix}$$

The full list of all logic gates supported in QISKIT is here: (This is actually the source code that defines the gates inside QISKIT):

https://github.com/Qiskit/qiskit-terra/blob/master/qiskit/qasm/libs/qelib1.inc


## "Running" a quantum circuit - the IBMQ Backends

Now that we have seen how to define a quantum circuit, we will see how to implement it using QISKIT. There are two different ways to "run" the circuit; on a classical computer simulating a perfect quantum computer, or on a quantum device itself. QISKIT calls these devices "backends". QISKIT allows us to send our circuit to the backend (as a QASM file - see my IBM Quantum Experience notes for an intro to QASM) and receive the results of the measurements (or simulated measurements).

Let us create a new circuit, whose output we know, to test it:

 

In [ ]:
# Create a Quantum Register with 5 qubits
qr_new = QuantumRegister(1)

# Create a Classical Register with 5 bits
cr_new= ClassicalRegister(1)

# Create a Quantum Circuit acting on the qr and cr register
circuit_new = QuantumCircuit(qr_new, cr_new)

circuit_new.h(qr_new[0])
circuit_new.measure(qr_new,cr_new)





In [ ]:
circuit_drawer(circuit_new)

This circuit creates the superposition state $(|0\rangle+|1\rangle)/\sqrt{2}$, and when we measure it, we will find the output '0' with probablility 1/2 and '1' with probability half.

## The classical simulator "Aer"

IBM calls its classical quantum computer simulator Aer. To see the reason for the name: https://medium.com/qiskit/qiskit-and-its-fundamental-elements-bcd7ead80492

We will use it to simulate our test circuit. First we need to install a module and initialise the simulator backend:


In [ ]:
from qiskit import Aer
backend = Aer.get_backend('qasm_simulator')

Let us create a job to send our circuit to this backend:

In [ ]:
# Create a Quantum Program for execution 
job = execute(circuit_new, backend)

We can check the status of our job.

In [ ]:
job.status()

Now that it has run, let's take a look at the results.

This is not the most elegant aspect of QISKIT. First, we must create a result object, then we run the `.get_counts` method (**Important**: note that the `.get_counts` method takes the name of the circuit as its argument)

In [ ]:
result = job.result()
result.get_counts(circuit_new)

We see that our simulator has simulated the circuit 1024 times (this number can be changed via the `shots` parameter in execute), in approximately half of the cases we get 0 and approximately half we get 1. These are the results we'd expect.

Note that the results have the form of a Python dictionary.

## Running the circuit on the real chip

To run a circuit on a real chip, IBM requires that you have an account. Each user has a certain number of credits which enable a limited number of runs per day on the hardware. Log on to your account:

https://quantum-computing.ibm.com

Log in and you should see your API key in a box on the main page. You can copy your API Key from here.

You now need to save that on your computer. You can do by pasting your key into the command box below and then executing that and the following commands.



In [ ]:
mykey='PASTE_YOUR_API_KEY_HERE'

In [ ]:
from qiskit import IBMQ
 
IBMQ.save_account(mykey)

Now that your key is saved, on your computer, you can access it (and enable IBMQ chip access) with the simple command":

In [ ]:
IBMQ.load_account()

You will need to save your key separately on any computer you use.

Now we can check the devices available to us. IBMQ uses something called a `provider` to specify which chips and simulations we have access to.

In [ ]:
IBMQ.providers()

There should be just one provider available to you. You can activate it by creating a `provider` object which we will call simply 'provider'. We can then ask the provider object to tell us which backends are available to us. 

A backend is either a simulator or a physical chip. If you have played around with IBM Q Experience webpages these backends should be familiar to you.

In [ ]:
provider = IBMQ.get_provider(hub='ibm-q')
provider.backends()

The following script (adapted from the IBMQ tutorial) allows us to select the least busy 5-qubit backend for our computation and save it as the python object `backend`.

In [ ]:
from qiskit.providers.ibmq import least_busy

small_devices = provider.backends(filters=lambda x: x.configuration().n_qubits == 5
                                   and not x.configuration().simulator)
backend=least_busy(small_devices)

print(backend)

For more details on the above example, and on the options available in `provider` see IBM's tutorial here: https://github.com/Qiskit/qiskit-ibmq-provider/blob/master/docs/tutorials/1_the_ibmq_account.ipynb

Now the syntax to run our code on the real chip is exactly the same as for the simulator.

In [ ]:
job = execute(circuit_new, backend)

In [ ]:
job.status()

In [ ]:
result = job.result()
result.get_counts(circuit_new)

For more details on running jobs, and in particular using the **job manager** to track jobs which take a while to execute see: 

https://qiskit.org/documentation/apidoc/ibmq-provider.html

and click **Job Manager**.


## Some useful tips for running jobs

### Job ID's
During busy periods, it may take some time (can be several hours) for your job to run. In this case, you need a way to check your job's status, and recover its results, perhaps the next day. Also, given the small number of ties you can execute circuits on the chip, the data from every run is sacred. **Job ID's** help you keep track of jobs and recover their results.

You can access the Job ID for a job, by using the `.job_id()` method on the job you have executed. E.g.:



In [ ]:
myjobid=job.job_id()

print('JOB ID: {}'.format(myjobid))  

We can use the job ID to retrieve a job object from our backend at a later time:

In [ ]:
job_retrieved=backend.retrieve_job(myjobid)

result_retrieved = job_retrieved.result()
result_retrieved.get_counts(circuit_new)

### Getting help

QISKIT documentation can be found here:

https://qiskit.org/documentation/

It is somewhat incomplete due to the rapid development of QISKIT. If you have a question that the documentation can't answer, I suggest you check the Quantum Computing Stackexchange site, which IBM uses as a main support outlet for QISKIT.

https://quantumcomputing.stackexchange.com
